In [2]:
import json
data = json.load(open('asl_quant.json'))

data.keys()

dict_keys(['input_1', 'quant_conv2d', 'quant_conv2d_1', 'quant_conv2d_1_relu', 'quant_conv2d_2', 'quant_conv2d_2_relu', 'quant_conv2d_3', 'quant_conv2d_3_relu', 'quant_conv2d_relu', 'quant_dense', 'quant_dense_1', 'quant_dense_1_relu', 'quant_dense_2', 'quant_dense_2_relu', 'quant_dense_3', 'quant_dense_3_softmax', 'quant_dense_relu', 'quant_flatten', 'quant_input_1', 'quant_max_pooling2d', 'quant_max_pooling2d_1', 'quant_max_pooling2d_2', 'top_level_model_weights'])

In [126]:
import numpy as np
output = {}


for key in data.keys():

    if ("input" not in key):
        splitted_key = key.split("_")
        layer_name = splitted_key[1]

        if (len(splitted_key)>2):

            if (len(splitted_key)<=3):

                if (splitted_key[2].isdigit()):

                    layer_name = layer_name + "_" + str(int(splitted_key[2])+1)
    
                    
                    if (("bias:0" not in data[key].keys()) and ("kernel:0" in data[key].keys())):
                        weight = data[key]["kernel:0"]

                        output[layer_name] = {"bias": list(np.zeros(np.array(weight).shape)),"weights":data[key]["kernel:0"]}
                    

                    elif ("bias:0" in data[key].keys() and "kernel:0" in data[key].keys()):
                        output[layer_name] = {"bias":data[key]["bias:0"], "weights":data[key]["kernel:0"]}

                
        else:
            layer_name = layer_name + "_1"
            
            if (("bias:0" not in data[key].keys()) and ("kernel:0" in data[key].keys())):
                weight = data[key]["kernel:0"]

                output[layer_name] = {"bias": list(np.zeros(np.array(weight).shape)),"weights":data[key]["kernel:0"]}
                
                    

            elif ("bias:0" in data[key].keys() and "kernel:0" in data[key].keys()):
                output[layer_name] = {"bias":data[key]["bias:0"], "weights":data[key]["kernel:0"]}



In [127]:
print(output.keys())

dict_keys(['conv2d_1', 'conv2d_2', 'conv2d_3', 'conv2d_4', 'dense_1', 'dense_2', 'dense_3', 'dense_4'])


In [128]:
output['conv2d_1']

{'bias': [-0.012814601883292198,
  -0.00400659441947937,
  -0.0025669937022030354,
  -0.0005987074691802263,
  -0.005565119907259941,
  0.0007364000775851309,
  0.0017828337149694562,
  -0.0036262848880141973,
  -0.009982417337596416,
  -0.004705503582954407,
  -0.006186370737850666,
  -0.012533756904304028,
  -0.013569221831858158,
  -0.001141739310696721,
  0.02562863752245903,
  -0.011000377126038074,
  0.007310651708394289,
  -0.002246537245810032,
  -0.0014385887188836932,
  0.0021010362543165684,
  -0.0002338178310310468,
  0.003954126965254545,
  -0.0009142059716396034,
  0.004172664135694504,
  -0.009675811044871807,
  -0.007602027151733637,
  0.015316563658416271,
  -0.002420616103336215,
  0.0052739037200808525,
  0.014278586953878403,
  7.057828042889014e-05,
  0.0005178609862923622],
 'weights': [[[[-0.11657049506902695,
     0.0024848885368555784,
     0.04576302692294121,
     0.059297677129507065,
     -0.023002760484814644,
     0.029236948117613792,
     0.046532355248

In [139]:
# open a new .h file and start writing the weights and biases

f = open("asl_weights.h", "w")

f.write("""#include <stdint.h> 

using namespace std;""")

for key in output.keys():
    weights = output[key]["weights"]
    bias = output[key]["bias"]

    weights_shape = np.array(weights).shape
    bias_shape = np.array(bias).shape

    #converting weights shape to a string
    weights_shape_str = str(weights_shape).replace("(","").replace(")","").replace(", ","*")
    bias_shape_str = str(bias_shape).replace("(","").replace(")","").replace(", ","*")


    weights = np.array(weights).reshape(-1)
    bias = np.array(bias).reshape(-1)   

    f.write("\n\nstatic const double " + key + "_weights[" + weights_shape_str + "] = {")
    for i in range(len(weights)):
        f.write(str(weights[i].round(5))+ ", ")
    f.write("};")

    f.write("\n\nstatic const double " + key + "_bias[" + bias_shape_str + "] = {")
    for i in range(len(bias)):
        f.write(str(bias[i].round(5)) + ", ")
    f.write("};")

f.close()



In [95]:
data['input_2'].keys()

KeyError: 'input_2'

# quant dense 1

In [17]:
import numpy as np

np.array(data['quant_dense_1']['bias:0']).shape

(128,)

In [18]:
data['quant_dense_1'].keys()

dict_keys(['bias:0', 'bias_max:0', 'bias_min:0', 'bias_pos:0', 'kernel:0', 'kernel_max:0', 'kernel_min:0', 'kernel_pos:0', 'optimizer_step:0'])

In [19]:
np.array(data['quant_dense_1']['kernel:0']).shape

(526, 128)

# quant conv2d

In [21]:
import numpy as np

np.array(data['quant_conv2d']['bias:0']).shape

(32,)

In [22]:
data['quant_conv2d'].keys()

dict_keys(['bias:0', 'bias_max:0', 'bias_min:0', 'bias_pos:0', 'kernel:0', 'kernel_max:0', 'kernel_min:0', 'kernel_pos:0', 'optimizer_step:0'])

In [24]:
import numpy as np

np.array(data['quant_conv2d']['kernel:0']).shape

(3, 3, 3, 32)

In [31]:
data['quant_conv2d'].keys()

dict_keys(['bias:0', 'bias_max:0', 'bias_min:0', 'bias_pos:0', 'kernel:0', 'kernel_max:0', 'kernel_min:0', 'kernel_pos:0', 'optimizer_step:0'])

In [30]:
import numpy as np

np.array(data['quant_conv2d']['bias:0']).shape

(32,)

In [25]:
import numpy as np

np.array(data['quant_conv2d_1']['kernel:0']).shape

(3, 3, 32, 64)

In [27]:
import numpy as np

np.array(data['quant_conv2d_1_relu'])

array({'optimizer_step:0': -1.0, 'post_activation_max:0': 0.71490478515625, 'post_activation_min:0': -0.7205339670181274, 'post_activation_pos:0': 7.0},
      dtype=object)

In [29]:
import numpy as np

np.array(data['quant_conv2d_2']['kernel:0']).shape

(3, 3, 64, 64)